In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

2023-10-24 15:17:20.174559: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['gender']
    return ecg_data, label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-10-24 15:17:25.477253: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38006 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [5]:
def conv_block(x, n, k, mp):
    x = tf.keras.layers.Conv1D(n, k, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    if mp:
        x = tf.keras.layers.MaxPooling1D(mp)(x)

    return x

input_layer = tf.keras.layers.Input(shape=(5000,12))
x = input_layer

x = conv_block(x, 16, 7, 2)
x = conv_block(x, 16, 5, 4)
x = conv_block(x, 32, 5, 2)
x = conv_block(x, 32, 5, 4)
x = conv_block(x, 64, 5, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)

x = tf.keras.layers.Conv1D(128, 12, padding = 'same', data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling1D()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

In [6]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

model_name = "cnn-gender"
output_path = make_checkpoint_dir("data/models", model_name)

print(f"Model: {model_name} Run Path: {output_path}")

In [7]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(output_path, save_best=True),
    tf.keras.callbacks.CSVLogger(f"data/models/{model_name}-history.csv")
]

history = model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/10


2023-10-24 15:17:30.020665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-10-24 15:17:30.909698: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-24 15:17:31.257657: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f18ad485fe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-24 15:17:31.257717: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2023-10-24 15:17:31.333040: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-24 15:17:31.824146: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

   7851/Unknown - 169s 20ms/step - loss: 0.5033 - accuracy: 0.7548 - auc: 0.8341

2023-10-24 15:20:15.403323: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 15189529740224460961
2023-10-24 15:20:15.403405: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14847708875366506006


INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-24_15-17-26/assets


...

7851/7851 [==============================] - 186s 24ms/step - loss: 0.3914 - accuracy: 0.8268 - auc: 0.9051 - val_loss: 0.4730 - val_accuracy: 0.7841 - val_auc: 0.8691 - lr: 0.0010
Epoch 9/10
7849/7851 [============================>.] - ETA: 0s - loss: 0.3862 - accuracy: 0.8291 - auc: 0.9078INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-24_15-17-26/assets


INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-24_15-17-26/assets


7851/7851 [==============================] - 184s 23ms/step - loss: 0.3862 - accuracy: 0.8291 - auc: 0.9078 - val_loss: 0.4696 - val_accuracy: 0.7859 - val_auc: 0.8697 - lr: 0.0010
Epoch 10/10
7850/7851 [============================>.] - ETA: 0s - loss: 0.3814 - accuracy: 0.8317 - auc: 0.9101INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-24_15-17-26/assets


INFO:tensorflow:Assets written to: data/models/cnn-gender-2023-10-24_15-17-26/assets


7851/7851 [==============================] - 188s 24ms/step - loss: 0.3814 - accuracy: 0.8317 - auc: 0.9101 - val_loss: 0.4827 - val_accuracy: 0.7774 - val_auc: 0.8658 - lr: 0.0010


In [8]:
model.save(f"data/models/{model_name}.keras")